In [14]:
import os
import re
import glob
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from utils import *

In [15]:
CSVs = []
for root, dirs, files in os.walk("Affinity Data"):
    for file in files:
        if file.endswith(".csv"):
             CSVs.append(os.path.join(root, file))

In [16]:
len(CSVs)

377

In [17]:
CSVs = []
for root, dirs, files in os.walk("Affinity Data"):
    for file in files:
        if file.endswith(".csv"):
             CSVs.append(os.path.join(root, file))
# CSVs.remove('Affinity Data/AAV2/New AAV2/AAV2 HEK Lysate  - A10 (T) Run 3 002.csv')
# CSVs.remove('Affinity Data/AAVA3/AAVA3_AAV2 SF9_F3 002.csv')
# CSVs.remove('Affinity Data/AAVX_2/Column Evaluation - AAVx 2ml SAS-AM012921.csv')
# CSVs.remove('Affinity Data/AAVX_2/AAVX Resin For NCTM - 1ml Column HETP and ASYM.csv')
# CSVs.remove('Affinity Data/AAVX/AAVX 0.5ML Run SAS102122 004_fl.csv')

In [18]:
frame = {'resin': [], 'serotype': [], 'file': [], 'Column Volume (mL)':[], 'Pure':[], 'Blank':[],
         'Elution pH': [],'Wash pH': [],'Equlibration pH': [],'Elution Conductivity': [],
         'Wash Conductivity': [],'Equilibration Conductivity': [], 'Sample Volume (mL)':[]}
for csv in CSVs:
    name = csv.split('/')[-1][:-4]
    resin, serotype = get_resin_and_serotype(name)
    if resin == 'U':
        resin = get_resin(name)
    if serotype == 'U':
        serotype = csv.split('/')[1]
    pure = is_pure(name)
    col_vol = get_column_volume(name)
    blank = is_blank(name)
    # print(csv)
    try:
        df = pd.read_csv(csv, skiprows = [0,1], delimiter='\t', encoding='utf_16', on_bad_lines='skip', low_memory=False)
        data_dict = load_useful_data(df)
    except Exception as e:
        print(e, csv)
    
    try:
        elution_ph, elution_cond = get_ph_and_cond_at_elution(df, data_dict)
        wash_ph, wash_cond = get_ph_and_cond_at_wash(df, data_dict)
        equilibration_ph, equilibration_cond = get_ph_and_cond_at_equilibration(df, data_dict)
        sample_volume = get_sample_volume(df, data_dict)
        frame['resin'].append(resin)
        frame['serotype'].append(serotype)
        frame['file'].append(name)
        frame['Pure'].append(pure)
        frame['Blank'].append(blank)
        frame['Column Volume (mL)'].append(col_vol[:-2])
        frame['Elution pH'].append(elution_ph)
        frame['Wash pH'].append(wash_ph)
        frame['Equlibration pH'].append(equilibration_ph)
        frame['Elution Conductivity'].append(elution_cond)
        frame['Wash Conductivity'].append(wash_cond)
        frame['Equilibration Conductivity'].append(equilibration_cond)
        frame['Sample Volume (mL)'].append(sample_volume)
    except Exception as e:
        print(e, csv)

    



data = pd.DataFrame(frame)
data[['Column Diameter (mm)', 'Coulmn Height (cm)']] = 'U'
# data.loc[['Elution pH','Wash pH','Equlibration pH','Elution Conductivity','Wash Conductivity','Equilibration Conductivity']] = 'U'

cannot convert float NaN to integer Affinity Data/AAV2/AAVx method with LG6 cleaning 001.csv
cannot convert float NaN to integer Affinity Data/AAVX_3/AAVX 0.5ml - AAV5 HEK elution 001.csv
'Logbook' is not in list Affinity Data/AAVX/AAVX 0.5ML Run SAS102122 004_fl.csv
4 Affinity Data/AAVX/AAVX 0.5ML Run SAS102122 004_fl.csv
cannot convert float NaN to integer Affinity Data/AAVX/AAVX Run 1 Pure SAS10112022 001.csv
cannot convert float NaN to integer Affinity Data/AAVX/AAVX 0.5ml - AAV5 HEK elution 001.csv
cannot convert float NaN to integer Affinity Data/AAV9_with_LigaGuard/ethanol column cleaning 001.csv
cannot convert float NaN to integer Affinity Data/AAV9_with_LigaGuard/LigaGuard AAV fixed volume 50 mL load 001.csv
cannot convert float NaN to integer Affinity Data/AAV9_with_LigaGuard/LigaGuard AAV 50 mL 002.csv
cannot convert float NaN to integer Affinity Data/AAV9_with_LigaGuard/LigaGuard AAV 50 mL 001.csv
cannot convert float NaN to integer Affinity Data/AAV9/AAVx 9th cycle 50 mL l

In [25]:
data

,resin,serotype,file,Column Volume (mL),Pure,Blank,Elution pH,Wash pH,Equlibration pH,Elution Conductivity,Wash Conductivity,Equilibration Conductivity,Sample Volume (mL),Column Diameter (mm),Coulmn Height (cm)
0,AAVx,AAV2,AAVx method LG6 toyo Load 31.5 mL cycle 1 001,0.5,False,False,7.43,7.55,7.44,16.14,15.85,7.44,10.009468,5,2.55
1,AAVx,AAV2,AAVx method with LG6 Load 32 mL S2 cycle 8 001,0.5,False,False,7.66,7.51,2.05,16.27,14.35,2.05,10.009404,5,2.55
2,AAVx,AAV2,AAVx method with LG6 Load cycle 3 001,0.5,False,False,2.22,7.34,7.38,38.60,16.27,7.38,5.038723,5,2.55
3,AAVx,AAV2,AAVx method with LG6 Load 33 mL cycle 10 001,0.5,False,False,7.58,7.48,2.19,16.50,15.16,2.19,10.009544,5,2.55
4,AAVx,AAV2,AAVx method no LG6 Load 35 mL cycle 2 001,0.5,False,False,7.41,7.58,7.40,16.19,13.44,7.40,10.009632,5,2.55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359,AAVW8,AAV10,AAVW8_AAV10 HEK lysate 003,0.5,False,False,7.45,7.32,6.13,4.11,2.95,6.13,9.999899,5,2.55
360,AAVA5,AAV10,AAVA5_AAV10 HEK lysate 004,0.5,False,False,7.33,7.35,6.03,3.15,2.91,6.03,9.999840,5,2.55
361,AAVA1,AAV10,AAVA1_AAV10 HEK lysate 001,0.5,False,False,7.31,7.34,5.93,3.14,2.93,5.93,9.999899,5,2.55
362,AAVA2,AAV10,AAVA2_AAV10 HEK lysate 001,0.5,False,False,7.43,7.38,6.11,3.98,2.97,6.11,9.999947,5,2.55


In [26]:
for i in range(len(data)):
    if data['resin'][i] == 'U' and (data['serotype'][i] in ['AAV2', 'AAV6', 'AAV9', 'AAV9_with_LigaGuard']):
        # data['resin'][i] = 'AAVX'
        data.loc[i, 'resin'] = 'AAVX'

In [21]:
data

,resin,serotype,file,Column Volume (mL),Pure,Blank,Elution pH,Wash pH,Equlibration pH,Elution Conductivity,Wash Conductivity,Equilibration Conductivity,Sample Volume (mL),Column Diameter (mm),Coulmn Height (cm)
0,AAVx,AAV2,AAVx method LG6 toyo Load 31.5 mL cycle 1 001,,False,False,7.43,7.55,7.44,16.14,15.85,7.44,10.009468,U,U
1,AAVx,AAV2,AAVx method with LG6 Load 32 mL S2 cycle 8 001,,False,False,7.66,7.51,2.05,16.27,14.35,2.05,10.009404,U,U
2,AAVx,AAV2,AAVx method with LG6 Load cycle 3 001,,False,False,2.22,7.34,7.38,38.60,16.27,7.38,5.038723,U,U
3,AAVx,AAV2,AAVx method with LG6 Load 33 mL cycle 10 001,,False,False,7.58,7.48,2.19,16.50,15.16,2.19,10.009544,U,U
4,AAVx,AAV2,AAVx method no LG6 Load 35 mL cycle 2 001,,False,False,7.41,7.58,7.40,16.19,13.44,7.40,10.009632,U,U
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359,AAVW8,AAV10,AAVW8_AAV10 HEK lysate 003,,False,False,7.45,7.32,6.13,4.11,2.95,6.13,9.999899,U,U
360,AAVA5,AAV10,AAVA5_AAV10 HEK lysate 004,,False,False,7.33,7.35,6.03,3.15,2.91,6.03,9.999840,U,U
361,AAVA1,AAV10,AAVA1_AAV10 HEK lysate 001,,False,False,7.31,7.34,5.93,3.14,2.93,5.93,9.999899,U,U
362,AAVA2,AAV10,AAVA2_AAV10 HEK lysate 001,,False,False,7.43,7.38,6.11,3.98,2.97,6.11,9.999947,U,U


In [27]:
for i in range(len(data)):
    if data['Column Volume (mL)'][i] == '':
        data.loc[i, ['Coulmn Height (cm)', 'Column Diameter (mm)', 'Column Volume (mL)']] = [2.55, 5, 0.5]

In [28]:
data

,resin,serotype,file,Column Volume (mL),Pure,Blank,Elution pH,Wash pH,Equlibration pH,Elution Conductivity,Wash Conductivity,Equilibration Conductivity,Sample Volume (mL),Column Diameter (mm),Coulmn Height (cm)
0,AAVx,AAV2,AAVx method LG6 toyo Load 31.5 mL cycle 1 001,0.5,False,False,7.43,7.55,7.44,16.14,15.85,7.44,10.009468,5,2.55
1,AAVx,AAV2,AAVx method with LG6 Load 32 mL S2 cycle 8 001,0.5,False,False,7.66,7.51,2.05,16.27,14.35,2.05,10.009404,5,2.55
2,AAVx,AAV2,AAVx method with LG6 Load cycle 3 001,0.5,False,False,2.22,7.34,7.38,38.60,16.27,7.38,5.038723,5,2.55
3,AAVx,AAV2,AAVx method with LG6 Load 33 mL cycle 10 001,0.5,False,False,7.58,7.48,2.19,16.50,15.16,2.19,10.009544,5,2.55
4,AAVx,AAV2,AAVx method no LG6 Load 35 mL cycle 2 001,0.5,False,False,7.41,7.58,7.40,16.19,13.44,7.40,10.009632,5,2.55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359,AAVW8,AAV10,AAVW8_AAV10 HEK lysate 003,0.5,False,False,7.45,7.32,6.13,4.11,2.95,6.13,9.999899,5,2.55
360,AAVA5,AAV10,AAVA5_AAV10 HEK lysate 004,0.5,False,False,7.33,7.35,6.03,3.15,2.91,6.03,9.999840,5,2.55
361,AAVA1,AAV10,AAVA1_AAV10 HEK lysate 001,0.5,False,False,7.31,7.34,5.93,3.14,2.93,5.93,9.999899,5,2.55
362,AAVA2,AAV10,AAVA2_AAV10 HEK lysate 001,0.5,False,False,7.43,7.38,6.11,3.98,2.97,6.11,9.999947,5,2.55


In [29]:
data.to_csv('affinity_data.csv', index=False)

In [50]:
data['serotype'].unique()

array(['AAV2', 'AAVA3', 'AAV9', 'AAV1', 'AAV6', 'AAV10', 'AAV7', 'AAV4',
       'AAV5', 'AAV3', 'AAV8', 'AAVX_3', 'AAVX', 'AAV9_with_LigaGuard',
       'AAVX_2'], dtype=object)

In [ ]:
['Ligand ID', 'Serotype', 'Equil Residence Vol', 'Equil Conductivity', 'Equil pH', 'Elution Residence Vol' 'Elution Conductivity', 'Elution pH', 'Wash Residence Vol', 'Wash pH', 'Wash Conductivity']

In [ ]:
cannot convert float NaN to integer Affinity Data/AAV2/AAVx method with LG6 cleaning 001.csv
cannot convert float NaN to integer Affinity Data/AAVX_3/AAVX 0.5ml - AAV5 HEK elution 001.csv
cannot convert float NaN to integer Affinity Data/AAVX/AAVX Run 1 Pure SAS10112022 001.csv
cannot convert float NaN to integer Affinity Data/AAVX/AAVX 0.5ml - AAV5 HEK elution 001.csv
cannot convert float NaN to integer Affinity Data/AAV9_with_LigaGuard/ethanol column cleaning 001.csv
cannot convert float NaN to integer Affinity Data/AAV9_with_LigaGuard/LigaGuard AAV fixed volume 50 mL load 001.csv
cannot convert float NaN to integer Affinity Data/AAV9_with_LigaGuard/LigaGuard AAV 50 mL 002.csv
cannot convert float NaN to integer Affinity Data/AAV9_with_LigaGuard/LigaGuard AAV 50 mL 001.csv
cannot convert float NaN to integer Affinity Data/AAV9/AAVx 9th cycle 50 mL load 001.csv
cannot convert float NaN to integer Affinity Data/AAVX_2/AAVX HEK293 2mL column method(637483396913297566).csv
cannot convert float NaN to integer Affinity Data/AAVX_2/AAVX HEK293 0.1mL column DBC Study Part I - 033121 001 - 050721 JAH 001.csv